In [ ]:
pip install numpy pandas networkx

In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import math
import matplotlib.pyplot as plt

In [2]:
def printPriorities(input_headers_vertical, input_headers_horizontal, input_data):
    print(pd.DataFrame(input_data, input_headers_vertical, input_headers_horizontal))

In [3]:
# №_11.7. Групповая оценка приоритетов

In [4]:
# №_11.7.1. Усреднение индивидуальных приоритетов
# input_file_path - путь к csv файлу
def calculate_averaging_individual_priorities(input_file_path):
    # Считываем данные из csv файла:
    #    A, B, C, D, Эксперт
    #    1, 1, 1, 1, 1
    #    0, 1, 1, 1, 1
    #    0, 0, 1, 1, 1
    #    0, 0, 0, 1, 1
    input_data = pd.read_csv(input_file_path)
    input_data_length = input_data.shape[1] - 1
    input_data = input_data.sort_values(by=[input_data.columns.values[input_data_length]])

    # Считываем названия объектов сравнения
    input_headers = input_data.columns.values[:input_data_length].tolist()
    
    # Считаем количество экспертов
    number_of_experts = len(set(input_data[input_data.columns[input_data_length]]
                                .to_numpy()
                                .flatten()))
    
    # Получаем группы оценок для каждого эксперта
    priorities = np.array_split(
        input_data[input_data.columns[:input_data_length]].to_numpy(),
        number_of_experts
    )

    # Считаем, что важность каждого эксперта одинаковая - в пособии приводятся только такие примеры
    # При необходимости можно доработать скрипт
    competence = 1 / number_of_experts
    
    # Проходимся по алгоритму и считаем данные
    objects_count_i = np.array(priorities).shape[0]
    objects_count_j = np.array(priorities).shape[1]
    group_priorities = []
    
    # Находим сумму w_i
    for priority in priorities:
        total_sum = np.sum(priority)
        group_priority = []

        for row in priority:    
            number_of_ones = np.sum(row)
            normalized_priority = number_of_ones / total_sum
            group_priority += [normalized_priority]
        
        group_priorities += [group_priority]

    w = np.zeros(objects_count_j)
    s_2 = np.zeros(objects_count_j)
    s = np.zeros(objects_count_j)
    u = np.zeros(objects_count_j)
    
    # Находим значение w
    for i in range(objects_count_i):
        for j in range(objects_count_j):
            w[j] += group_priorities[i][j]

    w *= competence

    # Находим s^2 - мера согласованности
    for i in range(objects_count_i):
        for j in range(objects_count_j):
            s_2[j] += (w[j] - group_priorities[i][j])**2
            
    s_2 *= competence
    s = s_2**0.5
    v = s / w
    
    print('\nУсреднение индивидуальных приоритетов')
    printPriorities(["w", "s_2", "s", "v"], input_headers, [w, s_2, s, v])

# Пример работы
calculate_averaging_individual_priorities('examples/example_1_1.csv')


Усреднение индивидуальных приоритетов
            A         B       C         D
w    0.325000  0.200000  0.2500  0.225000
s_2  0.006875  0.005000  0.0025  0.006875
s    0.082916  0.070711  0.0500  0.082916
v    0.255125  0.353553  0.2000  0.368514


In [5]:
# №_11.7.2. Расчёт приоритетов на основе групповых предпочтений
# input_file_path - путь к csv файлу
def calculate_group_priorities(input_file_path):
    # Считываем данные из csv файла:
    #    A, B, C, D
    #    4, 3, 2, 4
    #    1, 4, 1, 2
    #    2, 3, 4, 1
    #    0, 2, 3, 4
    input_data = pd.read_csv(input_file_path)

    # Считываем названия объектов сравнения
    input_headers = input_data.columns.values.tolist()
    
    # Получаем группы оценок для каждого эксперта
    priorities = input_data.to_numpy()
    
    # Проходимся по алгоритму и считаем данные
    objects_count_i = np.array(priorities).shape[0]
    objects_count_j = np.array(priorities).shape[1]

    group_priorities = []
    total_sum = np.sum(priorities)

    # Находим значение w
    for row in priorities:    
        number_of_ones = np.sum(row)
        normalized_priority = number_of_ones / total_sum
        group_priorities += [normalized_priority]

    w = np.zeros(objects_count_j)

    for j in range(objects_count_j):
        w[j] += group_priorities[j]        
    
    print('\nРасчет приоритетов на основе групповых предпочтений')
    printPriorities(["w"], input_headers, [w])


# Пример работы №_1
calculate_group_priorities('examples/example_2_1.csv')

# Пример работы №_2
calculate_group_priorities('examples/example_2_2.csv')


Расчет приоритетов на основе групповых предпочтений
       A    B     C      D
w  0.325  0.2  0.25  0.225

Расчет приоритетов на основе групповых предпочтений
       A     B     C      D
w  0.375  0.25  0.25  0.125


In [6]:
# №_11.7.3. Определение порядковой согласованности
# input_file_path - путь к csv файлу
def determining_ordinal_consistency(input_file_path):
    # Считываем данные из csv файла:
    #    A, B, C, D
    #    4, 3, 2, 4
    #    1, 4, 1, 2
    #    2, 3, 4, 1
    #    0, 2, 3, 4
    input_data = pd.read_csv(input_file_path)

    # Считываем названия объектов сравнения
    input_headers = input_data.columns.values.tolist()
    
    # Получаем группы оценок для каждого эксперта
    priorities = input_data.to_numpy()
    
    # Проходимся по алгоритму и считаем данные
    objects_count_i = np.array(priorities).shape[0]
    objects_count_j = np.array(priorities).shape[1]

    matrix = []
    graph_nodes = []

    # Строим матрицу смежности
    for i in range(objects_count_i):
        matrix_row = []
        for j in range(objects_count_j):
            if i == j:
                matrix_row += [0]
            elif priorities[i][j] >= priorities[j][i]:
                matrix_row += [1]
                graph_nodes += [(i, j)]
            else:
                matrix_row += [0]
        matrix += [matrix_row]    
    
    # Строим граф по матрице
    G = nx.DiGraph(graph_nodes)

    # Находим все циклы из 3 узлов (см. Микони - стр.45, формула - 2.9)
    cycles = sorted(nx.simple_cycles(G))
    filtered_cycles = sorted(filter(lambda c: len(c) == 3, cycles))
    cycles_count = len(filtered_cycles)
    max_cycles = math.comb(objects_count_j, 3)
    
    result = 1 - cycles_count / max_cycles
    
    print('\nОпределение порядковой согласованности')
    print("n: " + str(result))

# Пример работы
determining_ordinal_consistency('examples/example_3_1.csv')


Определение порядковой согласованности
n: 0.5


In [7]:
# №_11.7.4. Оценка согласия экспертов
# input_file_path - путь к csv файлу
def evaluation_of_expert_consensus(input_file_path):
    # Считываем данные из csv файла:
    #    A, B, C, D
    #    4, 3, 2, 4
    #    1, 4, 1, 2
    #    2, 3, 4, 1
    #    0, 2, 3, 4
    input_data = pd.read_csv(input_file_path)

    # Считываем названия объектов сравнения
    input_headers = input_data.columns.values.tolist()
    
    # Получаем группы оценок для каждого эксперта
    priorities = input_data.to_numpy()
    
    # Проходимся по алгоритму и считаем данные
    objects_count_i = np.array(priorities).shape[0]
    objects_count_j = np.array(priorities).shape[1]
    
    experts_count = objects_count_i
    s = 0

    # Общая степень согласия экспертных оценок s рассчитыется 
    # как сумма неравенств, сформированная для верхней треугольной матрицы в МПС:
    for i in range(objects_count_i - 1):
        for j in range(i + 1, objects_count_j):
            if i != j:
                s += abs(priorities[i][j] - priorities[j][i])

    S = experts_count * math.comb(objects_count_j, 2)
    
    result = s/S
    
    print('\nОценка согласия экспертов')
    print("u: " + str(result))
        
# Пример работы №_1
evaluation_of_expert_consensus('examples/example_4_1.csv')

# Пример работы №_2
evaluation_of_expert_consensus('examples/example_4_2.csv')

# Пример работы №_3
evaluation_of_expert_consensus('examples/example_4_3.csv')


Оценка согласия экспертов
u: 1.0

Оценка согласия экспертов
u: 0.4166666666666667

Оценка согласия экспертов
u: 0.5833333333333334


In [8]:
# №_11.8. Расчёт групповых рейтингов
# input_file_path - путь к csv файлу
def calculation_of_group_ratings(input_file_path):
    # Считываем данные из csv файла:
    #    A, B, C, D
    #    4, 3, 2, 4
    #    1, 4, 1, 2
    #    2, 3, 4, 1
    #    0, 2, 3, 4
    input_data = pd.read_csv(input_file_path)

    # Считываем названия объектов сравнения
    input_headers = input_data.columns.values.tolist()
    
    # Получаем группы оценок для каждого эксперта
    priorities = input_data.to_numpy()
    
    # Проходимся по алгоритму и считаем данные
    objects_count_i = np.array(priorities).shape[0]
    objects_count_j = np.array(priorities).shape[1]
    
    experts_count = objects_count_i

    r = np.zeros(objects_count_j)
    r_ = np.zeros(objects_count_j)
    d = np.zeros(objects_count_j)
    d_2 = np.zeros(objects_count_j)

    # Суммарный рейтинг
    for i in range(objects_count_i):
        for j in range(objects_count_j):
            r[j] += priorities[i][j]
    
    # Средневзвешенный ранг объектов
    for j in range(objects_count_j):
        r_[j] = round(r[j] / experts_count)
        
    # Средний суммарный рейтинг
    mid_r = 0.5 * experts_count * (objects_count_j + 1)

    # Расстояние
    d = r - mid_r
    d_2 = d**2
    
    s_d_2 = 0
    
    # Мера согласованности n рейтингов
    for j in range(objects_count_j):
        s_d_2 += d_2[j]
    
    print('\nРасчет групповых рейтингов')
    printPriorities(["r", "r'", "d", "d_2"], ["A", "B", "C", "D", "E", "F"], [r, r_, d, d_2])
    
    print("s: " + str(s_d_2))
    
    # Строим расчеты для максимального значения S

    max_r = np.zeros(objects_count_j)
    max_r_ = np.zeros(objects_count_j)
    max_d = np.zeros(objects_count_j)
    max_d_2 = np.zeros(objects_count_j)

    for j in range(objects_count_j):
        max_r[j] = (j + 1) * experts_count
    
    for j in range(objects_count_j):
        max_r_[j] = round(max_r[j] / experts_count)

    max_d = max_r - mid_r
    max_d_2 = max_d**2
    
    max_s_d_2 = 0
    
    for j in range(objects_count_j):
        max_s_d_2 += max_d_2[j]
    
    # Результаты
    printPriorities(["max_r", "max_r'", "max_d", "max_d_2"], ["A", "B", "C", "D", "E", "F"], [max_r, max_r_, max_d, max_d_2])
    
    print("max_S: " + str(max_s_d_2))

    print("\nW: " + str(s_d_2/max_s_d_2))

# Пример работы
calculation_of_group_ratings('examples/example_5_1.csv')



Расчет групповых рейтингов
        A     B     C     D     E     F
r    15.0  11.0  10.0  19.0  12.0  17.0
r'    4.0   3.0   2.0   5.0   3.0   4.0
d     1.0  -3.0  -4.0   5.0  -2.0   3.0
d_2   1.0   9.0  16.0  25.0   4.0   9.0
s: 64.0
             A     B     C     D     E      F
max_r      4.0   8.0  12.0  16.0  20.0   24.0
max_r'     1.0   2.0   3.0   4.0   5.0    6.0
max_d    -10.0  -6.0  -2.0   2.0   6.0   10.0
max_d_2  100.0  36.0   4.0   4.0  36.0  100.0
max_S: 280.0

W: 0.22857142857142856


In [9]:
# №_11.10. Согласованность оценок на шкале качества
# input_file_path - путь к csv файлу
# number_of_indicators - количество показателей
# number_of_experts - количество экспертов
def consistency_of_scores_on_the_quality_scale(input_file_path, number_of_indicators, number_of_experts):
    # Считываем данные из csv файла:
    #     Объект, ОВ, В, С, Н
    #     № 14,  10, 0,  0, 5
    #     № 6,   5,  10, 0, 0
    #     № 15,  5,  0,  0, 10
    #     № 10,  4,  4,  4, 3
    input_data = pd.read_csv(input_file_path)

    # Считываем названия объектов сравнения
    input_headers = input_data.columns.values.tolist()
    
    # Получаем группы оценок для каждого эксперта
    priorities = input_data.iloc[: , 1:].to_numpy()
    object_names = input_data.iloc[: , 0].to_numpy()
    
    # Проходимся по алгоритму и считаем данные
    objects_count_i = np.array(priorities).shape[0]
    array_of_inconsistency = []

    for i in range(objects_count_i):
        objects_count_j = len(np.array(priorities[i]))
        inconsistency = 0

        for z in range(objects_count_j - 1):
            for v in range(z + 1, objects_count_j):
                inconsistency += priorities[i][z] * priorities[i][v] * abs(z - v)
        
        array_of_inconsistency += [inconsistency]
    
    # Количество категорий
    m = np.array(priorities[i]).shape[0]

    # Количество показателей
    n = number_of_indicators
    
    # Количество экспертов
    M = number_of_experts
    
    # Максимальное значение согласованности
    temp_s_max = (M * n / 2) // 1
    S_max = temp_s_max * (M * n - temp_s_max) * (m - 1)
    
    result_array = []
    numeration = []
    
    max_value = -1
    max_label = ""
    
    # Находим значение с наибольшей согласованностью
    for i in range(objects_count_i):
        u_i = 1 - array_of_inconsistency[i] / S_max
        result_array += [[object_names[i], u_i]]
        numeration += [str(i)]
        
        if max_value < u_i:
            max_value = u_i
            max_label = object_names[i]
        
    print('\nСогласованность оценок на шкале качества')
    printPriorities(numeration, ["Объект", "u(x_i)"], result_array)
    
    print('\nПредпочтение объекту ' + max_label + ': ' + str(max_value))

# Пример работы
consistency_of_scores_on_the_quality_scale('examples/example_6_1.csv', 3, 5)


Согласованность оценок на шкале качества
  Объект    u(x_i)
0   № 14  0.107143
1    № 6  0.702381
2   № 15  0.107143
3   № 10  0.190476

Предпочтение объекту № 6: 0.7023809523809523
